In [ ]:
!nvidia-smi


Tue Oct 28 10:07:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   67C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install -U transformers accelerate datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 22.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.


In [ ]:
from transformers import TrainingArguments
output_dir = "/content/drive/MyDrive/colab_task2/gpt2_recipe"


In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=1,   # fits on T4
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    save_total_limit=2,
    logging_steps=100,
    learning_rate=5e-5,
    fp16=True
)


In [ ]:
!nvidia-smi


Tue Oct 28 10:15:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P0             29W /   70W |     102MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os

# Set Kaggle credentials
os.environ['KAGGLE_USERNAME'] = "f258008ridamohsin"
os.environ['KAGGLE_KEY'] = "fac89010fb68851b28c69ff53481c3f7"

# Download dataset
!kaggle datasets download -d nazmussakibrupol/3a2mext

# Unzip into /content/data
!mkdir -p /content/data
!unzip -o 3a2mext.zip -d /content/data


Dataset URL: https://www.kaggle.com/datasets/nazmussakibrupol/3a2mext
License(s): unknown
 95% 467M/491M [00:02<00:00, 147MB/s] 
100% 491M/491M [00:02<00:00, 188MB/s]
Archive:  3a2mext.zip
  inflating: /content/data/3A2M_EXTENDED.csv  


In [2]:
from datasets import load_dataset, Dataset
from transformers import GPT2TokenizerFast
import pandas as pd
!ls /content/data


3A2M_EXTENDED.csv


In [3]:
data_path = "/content/data/3A2M_EXTENDED.csv"
df = pd.read_csv(data_path)
df.head()


,title,NER,Extended_NER,genre,label,directions
0,\t Arugula Pomegranate Salad,"[""baby spinach"", ""baby arugula"", ""pomegranate ...","['alfalfa sprouts', 'baby spinach', 'baby arug...",vegetables,4,"[""Toss together spinach and arugula, then plac..."
1,\t Black Bean And Turkey Chili,"[""olive oil"", ""yellow onion"", ""garlic"", ""groun...","['one', 'yellow onion', 'tomato paste', 'about...",sides,8,"[""Dice the onion and mince the garlic. Add the..."
2,\t Finger Lickin' Tofu Nuggets,"[""extra firm"", ""almond flour"", ""nutritional ye...","['extra firm', '2', 'coconut oil', 'almond flo...",nonveg,3,"[""Wrap the tofu in a clean tea towel and press..."
3,\t Jerk Beef Stew With Carrots And Tomatoes,"[""olive oil"", ""boneless beef chuck"", ""onion"", ...","['boneless beef chuck', '2', 'Saute', 'onion',...",vegetables,4,"[""Preheat oven to 350 degrees F."", ""Heat the o..."
4,\t Pomegranate Couscous Salad,"[""pomegranate arils"", ""whole wheat couscous"", ...","['whole wheat couscous', '10 minutes', 'lemon ...",vegetables,4,"[""Place couscous in a bowl with 11/2 cups of h..."


In [6]:
import pandas as pd
from datasets import Dataset

data_path = "/content/data/3A2M_EXTENDED.csv"
df = pd.read_csv(data_path)

In [7]:
from datasets import Dataset
import pandas as pd

# 🔹 Step 1: Load the CSV
data_path = "/content/data/3A2M_EXTENDED.csv"
df = pd.read_csv(data_path)

# 🔹 Step 2: Use a smaller subset (you can increase this later)
df = df.sample(800, random_state=42).reset_index(drop=True)
print("✅ Loaded dataset with", len(df), "samples")

# 🔹 Step 3: Combine columns safely (no .apply)
texts = []
for i, row in df.iterrows():
    title = str(row["title"]).strip().replace("\\t", "")
    ingredients = str(row["Extended_NER"])
    directions = str(row["directions"])
    texts.append(f"Title: {title}\nIngredients: {ingredients}\nDirections: {directions}")

# 🔹 Step 4: Convert to HuggingFace Dataset
dataset = Dataset.from_dict({"text": texts})

print("✅ Combined dataset ready — total samples:", len(dataset))
print(dataset[0])


✅ Loaded dataset with 800 samples
✅ Combined dataset ready — total samples: 800
{'text': 'Title: Jamaican Sweet Potato Pone\nIngredients: [\'30 minutes\', \'eggs\', \'brown sugar\', \'20 minutes\', \'Stir in the orange juice\', \'sweet potatoes\', \'kosher salt\', \'350\\\\u00b0F.\', \'Lightly beat the egg yolks\', \'dark rum\', \'butter\', \'Bake\', \'orange juice\', \'Cook\']\nDirections: ["Preheat the oven to 350\\u00b0F.", "Cook the sweet potato slices in boiling water for 20 minutes or until tender.", "Drain and mash with the brown sugar and butter.", "Stir in the orange juice.", "Lightly beat the egg yolks and add them to the mixture with the rum.", "Grease a pie plate.", "Beat the egg whites with the salt until stiff, but not dry.", "Gently fold the egg whites into the potato mix.", "Pour into the prpared pie plate.", "Bake for 30 minutes or until a knife inserted in the cener comes out clean."]'}


In [8]:
from transformers import GPT2TokenizerFast

# 🔹 Load GPT-2 tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# 🔹 GPT-2 doesn’t have a padding token by default
tokenizer.pad_token = tokenizer.eos_token

# 🔹 Tokenization function
def tokenize_batch(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=512,   # safe length for T4 GPU
        padding="max_length"
    )

# 🔹 Apply tokenization
tokenized_dataset = dataset.map(tokenize_batch, batched=True, remove_columns=["text"])

print("✅ Tokenization complete.")
print(tokenized_dataset[0])


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

✅ Tokenization complete.
{'input_ids': [19160, 25, 21661, 7490, 15335, 43876, 350, 505, 198, 41222, 25, 37250, 1270, 2431, 3256, 705, 33856, 82, 3256, 705, 33282, 7543, 3256, 705, 1238, 2431, 3256, 705, 1273, 343, 287, 262, 10912, 13135, 3256, 705, 34751, 18821, 3256, 705, 74, 38321, 8268, 3256, 705, 14877, 6852, 84, 405, 65, 15, 37, 2637, 11, 705, 15047, 306, 4405, 262, 5935, 331, 349, 591, 3256, 705, 21953, 7440, 3256, 705, 4360, 353, 3256, 705, 33, 539, 3256, 705, 43745, 13135, 3256, 705, 28937, 20520, 198, 13470, 507, 25, 14631, 6719, 25080, 262, 14361, 284, 13803, 59, 84, 405, 65, 15, 37, 33283, 366, 28937, 262, 6029, 21219, 24314, 287, 24372, 1660, 329, 1160, 2431, 393, 1566, 15403, 33283, 366, 35, 3201, 290, 30407, 351, 262, 7586, 7543, 290, 9215, 33283, 366, 1273, 343, 287, 262, 10912, 13135, 33283, 366, 15047, 306, 4405, 262, 5935, 331, 349, 591, 290, 751, 606, 284, 262, 11710, 351, 262, 7440, 33283, 366, 43887, 589, 257, 2508, 7480, 33283, 366, 34979, 262, 5935, 13216, 351, 2

In [9]:
!pip install -U transformers


In [10]:
from transformers import GPT2LMHeadModel, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Split data
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# Define training arguments (no evaluation_strategy)
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/colab_task2/gpt2_recipe",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    learning_rate=5e-5,
    fp16=True,
    push_to_hub=False,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

print("✅ Trainer ready!")


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


✅ Trainer ready!


In [11]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [12]:
!pip uninstall -y wandb


In [13]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_SILENT"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"

In [15]:
trainer.train()


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,2.487900


TrainOutput(global_step=180, training_loss=2.387972174750434, metrics={'train_runtime': 145.4924, 'train_samples_per_second': 9.897, 'train_steps_per_second': 1.237, 'total_flos': 376260526080000.0, 'train_loss': 2.387972174750434, 'epoch': 2.0})

In [18]:
#  Save fine-tuned model + tokenizer properly
trainer.save_model("/content/drive/MyDrive/colab_task2/gpt2_recipe")
tokenizer.save_pretrained("/content/drive/MyDrive/colab_task2/gpt2_recipe")

print("✅ Model saved successfully!")


✅ Model saved successfully!


In [19]:
from transformers import pipeline

# 🔹 Load your fine-tuned model from Drive
model_path = "/content/drive/MyDrive/colab_task2/gpt2_recipe"
generator = pipeline("text-generation", model=model_path, tokenizer="gpt2")

# 🔹 Try generating recipes for a few prompts
prompts = [
    "Title: Spicy Garlic Pasta\nIngredients: garlic, olive oil, spaghetti, chili flakes",
    "Title: Mango Smoothie\nIngredients: mango, yogurt, honey, ice cubes",
    "Title: Chicken Curry\nIngredients: chicken, coconut milk, curry powder, onions, garlic"
]

for p in prompts:
    print("🧾 Prompt:", p)
    print("-" * 80)
    output = generator(p, max_length=250, num_return_sequences=1, temperature=0.8, top_p=0.9)
    print(output[0]["generated_text"])
    print("=" * 120, "\n")


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=250) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


🧾 Prompt: Title: Spicy Garlic Pasta
Ingredients: garlic, olive oil, spaghetti, chili flakes
--------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=250) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Title: Spicy Garlic Pasta
Ingredients: garlic, olive oil, spaghetti, chili flakes, oregano
Directions: ["Cut spaghetti into bite sized pieces.", "In a small bowl, combine spaghetti, olive oil, garlic, and oregano.", "Add sausage, and toss with spaghetti.", "Sprinkle with pasta."]
Directions: ["Put spaghetti pieces in large saucepan.", "Cover and simmer for 3 to 4 minutes.", "Remove from heat and stir in garlic and tomatoes.", "Drain pasta and season with salt and pepper.", "Transfer pasta to a wire rack and refrigerate overnight. Store pasta in a warm place in a covered dish for up to 1 week."]
Ingredients: ["Bake pasta in preheated 400\u00b0 oven at 400\u00b0 for 20 to 25 minutes.", "Cook pasta at 400\u00b0 for 3 to 4 minutes or until golden brown.", "Remove from heat and let cool slightly, then stir in pasta.", "Cover and simmer for 3 to 4 minutes or until pasta is tender.", "Drain pasta and season with salt and pepper."]
Ingredients: ["Mix remaining ingredients and place in plastic 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=250) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Title: Mango Smoothie
Ingredients: mango, yogurt, honey, ice cubes, vinegar, and sugar
Directions: Combine all ingredients except milk and juice in blender and blend until smooth. Add remaining ingredients, and blend until smooth. Add to ice cream mixture and smooth. Let cool. Store in refrigerator or freezer.
Directions: Place mango mixture in freezer and mix until smooth. Add mango slices, mix well, and mix well. Transfer to a small bowl and freeze at room temperature. Store in refrigerator or freezer.
Directions: Combine mango, yogurt, honey, ice cubes, and vinegar in blender and blend until smooth. Add to ice cream mixture and blend well. Let cool. Store in refrigerator or freezer.
Directions: Combine mango, yogurt, honey, ice cubes, and vinegar in blender until smooth. Add to ice cream mixture and blend well. Let cool. Store in refrigerator or freezer.
Directions: Combine mango, yogurt, honey, ice cubes, and vinegar in blender until smooth. Add to ice cream mixture and blend well.

In [20]:
!pip install gradio -q


In [21]:
import gradio as gr
from transformers import pipeline

# Load your fine-tuned model
model_path = "/content/drive/MyDrive/colab_task2/gpt2_recipe"
generator = pipeline("text-generation", model=model_path, tokenizer=model_path)

# Function to generate a recipe
def generate_recipe(title, ingredients):
    prompt = f"Title: {title}\nIngredients: {ingredients}\nDirections:"
    output = generator(prompt, max_length=250, num_return_sequences=1, temperature=0.8, top_p=0.9)
    recipe = output[0]["generated_text"]
    return recipe

# Create the Gradio interface
demo = gr.Interface(
    fn=generate_recipe,
    inputs=[
        gr.Textbox(label="Recipe Title", placeholder="e.g., Spicy Garlic Pasta"),
        gr.Textbox(label="Ingredients", placeholder="e.g., garlic, olive oil, spaghetti, chili flakes")
    ],
    outputs=gr.Textbox(label="Generated Recipe"),
    title="🍳 GPT-2 Recipe Generator",
    description="Enter a title and ingredients to generate a full recipe!"
)

# Launch the app
demo.launch()


Device set to use cuda:0


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d87f9d1534ea994f12.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [22]:
from huggingface_hub import login

login()


In [25]:
from huggingface_hub import HfApi, HfFolder
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# ✅ Load your fine-tuned model and tokenizer
model_path = "/content/drive/MyDrive/colab_task2/gpt2_recipe"
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

# ✅ Choose a repo name (it will appear as: https://huggingface.co/yo)
repo_name = "gpt2-recipe-generator"
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

print(f"✅ Model uploaded successfully! Check it at https://huggingface.co/{repo_name}")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...699rqs3/model.safetensors:   0%|          |  549kB /  498MB            

README.md: 0.00B [00:00, ?B/s]

✅ Model uploaded successfully! Check it at https://huggingface.co/gpt2-recipe-generator


In [26]:
import gradio as gr
from transformers import pipeline

# 🔹 Load your model from Hugging Face
generator = pipeline("text-generation", model="ridamohsin/gpt2-recipe-generator")

# 🔹 Define the recipe generation function
def generate_recipe(prompt):
    output = generator(
        prompt,
        max_length=250,
        num_return_sequences=1,
        temperature=0.8,
        top_p=0.9,
        do_sample=True
    )[0]["generated_text"]
    return output

# 🔹 Gradio interface
interface = gr.Interface(
    fn=generate_recipe,
    inputs=gr.Textbox(label="Enter ingredients or a dish name"),
    outputs=gr.Textbox(label="Generated Recipe"),
    title="🍳 GPT-2 Recipe Generator",
    description="Enter a dish name or ingredients, and get a creative recipe!"
)

interface.launch(share=True)


config.json:   0%|          | 0.00/874 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

Device set to use cuda:0


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c09cad45a1284e007a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [27]:
!pip install evaluate rouge_score nltk


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=1bd6fc707697926bc604c99122fc2f4d0299403a3594f4d434c6ec1a3aa42698
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [28]:
import evaluate
import nltk
from transformers import pipeline
from tqdm import tqdm

# Download NLTK tokenizer data (for BLEU)
nltk.download("punkt")

# Load your fine-tuned model
generator = pipeline("text-generation", model="ridamohsin/gpt2-recipe-generator")

# Load the same dataset (small sample for testing)
import pandas as pd

df = pd.read_csv("/content/data/3A2M_EXTENDED.csv").dropna().sample(10, random_state=42)
print("Using", len(df), "samples for quick evaluation")

# Initialize metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

preds, refs = [], []

# Generate recipes and compare with real ones
for _, row in tqdm(df.iterrows(), total=len(df)):
    prompt = f"Title: {row['title']}\nIngredients: {row['Extended_NER']}\nDirections:"
    generated = generator(prompt, max_length=200, num_return_sequences=1, temperature=0.8)[0]["generated_text"]

    preds.append(generated)
    refs.append(row["directions"])

# Compute BLEU & ROUGE
bleu_score = bleu.compute(predictions=preds, references=refs)
rouge_score = rouge.compute(predictions=preds, references=refs)

print("\n✅ Evaluation Results:")
print(f"BLEU Score: {bleu_score['bleu']:.4f}")
print(f"ROUGE-1: {rouge_score['rouge1']:.4f}")
print(f"ROUGE-2: {rouge_score['rouge2']:.4f}")
print(f"ROUGE-L: {rouge_score['rougeL']:.4f}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Device set to use cuda:0


Using 10 samples for quick evaluation


  0%|          | 0/10 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
100%|██████████| 10/10 [00:29<00:00,  2.91s/it]



✅ Evaluation Results:
BLEU Score: 0.0645
ROUGE-1: 0.2763
ROUGE-2: 0.0681
ROUGE-L: 0.1349
